In [1]:
# Read the Dataset called jeopardy using Pandas
import pandas as pd
jeopardy = pd.read_csv("jeopardy.csv")
jeopardy.head(5)
jeopardy.columns
# Some of the columns names have spaces in front. I will remove these spaces using a list comprehension
jeopardy.columns = [i.strip(" ") for i in jeopardy.columns]

OSError: File b'jeopardy.csv' does not exist

In [ ]:
jeopardy.columns

In [ ]:
jeopardy

In [ ]:
# Normalize the text columns(Question and Answer columns)
import string 
string.punctuation

In [ ]:
# Create a function that take in a string, convert it to lowercase and remove all punctuation in the string
# Then normalize the Question and Answer columns with the apply function.
def remove_punctuation(w):
    w = w.lower()
    for c in string.punctuation:
        w = w.replace(c,"")
    return w 
jeopardy["clean_question"] = jeopardy["Question"].apply(remove_punctuation)
jeopardy["clean_answer"] = jeopardy["Answer"].apply(remove_punctuation)

In [ ]:
jeopardy["clean_question"][1:5]
jeopardy["Value"]

In [ ]:
# Now is turn to normalize value column creating this function
def normalize(v):
    for c in string.punctuation:
        v = v.replace(c,"")
    try: 
        v = int(v)
    except Exception:
        v = 0
    return v 
jeopardy["clean_value"] = jeopardy["Value"].apply(normalize)

# Convert the Air Date column to a datetime column
pd.to_datetime(jeopardy["Air Date"])
print(jeopardy["clean_value"])

In [ ]:
# The variable match_count can interpreted as in how many responses, there are the same words between the questions and the responses
def jeopardy_row(row):
    split_answer = row["clean_answer"].split(" ")
    split_question = row["clean_question"].split(" ")
    match_count = 0
    if "the" in split_answer:
        split_answer.remove("the")
    if len(split_answer) == 0:
        return 0 
    for item in split_answer:
        if item in split_question:
            match_count +=1
    matches = match_count / len(split_answer)
    return matches 
jeopardy["answer_in_question"] = jeopardy.apply(jeopardy_row,axis=1)
jeopardy["answer_in_question"].mean()

In [ ]:
# In this step, I investigate how often new questions are repeats of older ones.


question_overlap = []
terms_used = set()
for index, row in jeopardy.iterrows():
    split_question = row["clean_question"].split(" ")
    # Use list comprehension to remove words with less than 5 characters
    split_question = [word for word in split_question if len(word) > 5]
    match_count = 0
    for word in split_question:
        if word in terms_used:
            match_count += 1 
    for word in split_question:
        terms_used.add(word)  
    # It divides match_count with len(split_question) and assign the result to match_count
    if len(split_question) > 0:
            match_count /= len(split_question)
    question_overlap.append(match_count)
jeopardy["question_overlap"] = question_overlap
jeopardy["question_overlap"].mean()


In [ ]:
# Analize low value vs high value question. First step, is to create a new column called  high value that takes the value of 1 
# in case of the value in clean_value column is higher than 800, and 0 otherwise.

def data_frame_row(row):
    if row["clean_value"] > 800:
        value = 1
    else:
        value = 0
    return value
jeopardy["high_value"] = jeopardy.apply(data_frame_row,axis=1)
print(jeopardy["high_value"])


In [ ]:
# Create a function with the aim to count how many times the word in split_question occur in low value questions and high value questios
def function_word(word):
    low_count = 0
    high_count = 0
    for index, row in jeopardy.iterrows():
        if word in row["clean_question"].split(" "): # no me habia dado cuenta de esto 
            if row["high_value"] == 1:
                high_count += 1
            else:
                low_count += 1
    return high_count, low_count 

observed_expected = []
comparison_terms= list(terms_used)

for item in comparison_terms:
    observed_expected.append(function_word(item))
    #counts= function_word(item)
    #observed_expected.append(counts)
print(observed_expected)
#print(comparison_terms)

In [ ]:
observed_expected = []
comparison_terms= list(terms_used)

for item in comparison_terms:
    observed_expected.append(function_word(item))
observed_expected


In [2]:
import numpy as np
from scipy.stats import chisquare

# Applying The Chi-Squared Test 
# Find the number of rows in jeopardy where high_value=1 and high_value=0 
high_value_count = len(jeopardy[jeopardy["high_value"] == 1])
low_value_count = len(jeopardy[jeopardy["high_value"] == 0])
chi_squared = []
for item in observed_expected:
    total = item[0] + item[1] 
    total_prop = total / jeopardy.shape[0]
    # Multiply total_prop by high_value_count to get expected term count for high value rows
    exp_high_values = total_prop * high_value_count
    exp_low_values = total_prop * low_value_count
    exp_values = (exp_high_values + exp_low_values) / 2 
    chisquare_value, p_value = chisquare(total,expected_values)
    chi_squared.append(chisquare_value)
chi_squared


NameError: name 'jeopardy' is not defined